This is the original notebook used to extract HARP parameters, associate them with DEM results for different time intervals, and plot average values for different NuSTAR observations. It uses the all_targets keys and the functions which fetch results per-key. This means that files are pulled without regard for flaring vs. non-flaring, and that determination is done as part of the analysis (slow).

HARP parameters are available every 12 minutes, meaning that distinguishing between flaring/non-flaring HARP values is not feasible for many cases in the NuSTAR sample (which is rich with few-to-ten minute microflares). 

Because of this, a different method of extracting HARP parameters is included in add_result_params.ipynb – using the function haa.add_harp_params(), relevant HARP values are saved to each key dictionary, along with lists of HARP-array indices that intersect in time with NuSTAR orbits. 

In [ ]:
#import drms
import numpy as np
from matplotlib import pyplot as plt
#import matplotlib.ticker as mtick
#from matplotlib.dates import *
#from datetime import datetime as dt_obj


#Path to top-level do-dem directory - edit for your system.
path_to_dodem = '/Users/jmdunca2/do-dem/'
from sys import path as sys_path
sys_path.append(path_to_dodem+'/dodem/')

import HARP_and_age as haa
import all_nu_analysis as ana
# import visualize_dem_results as viz

import pickle
import importlib

In [ ]:
"""
This takes the all_targets dictionary and for each orbit for each key, determines if the orbit times overlap with the times that 
the HARP patch is defined (where the HARP number is also taken from the all_targets key entry). 

It makes two lists: keys_with_harps, orbits_with_harps (which are what they sound like).

Optionally (if plot=True), it makes a plot (radial + LOS unsigned flux) for each key with a HARP patch.


"""


plot=False

importlib.reload(haa)

with open('all_targets.pickle', 'rb') as f:
    data = pickle.load(f)

keys_with_harps=[]
orbits_with_harps=[]

keys_to_check = ['03-may-21_1','20-jul-21']
#keys_to_check = data.keys()

#keys_with_bad_vibes=['03-may-21_1', '20-jul-21']



for kk in keys_to_check:
    print('')
    
    harps, orbits = haa.get_harp_and_times(kk, data)
    for hh in harps:
        keys, segments = c.query('hmi.sharp_720s['+str(hh)+'][? (QUALITY<65536) ?]', 
                                 key='T_REC, USFLUX, ERRVF, MEANALP, MEANJZH, TOTUSJZ, LON_FWT', seg='Br')
        if keys.empty: 
            print('No result for HARP ', hh, ' key: ', kk)
            continue
            
        t_rec = np.array([haa.parse_tai_string(keys.T_REC[i],datetime=True) for i in range(keys.T_REC.size)])

        for o in orbits:
            if haa.interval_intersect([t_rec[0], t_rec[-1]], o):
                print(kk+': HARP '+str(hh)+' in orbit: ', o[0].strftime('%H-%M-%S'), o[1].strftime('%H-%M-%S'))
                if kk not in keys_with_harps:
                    #if kk not in keys_with_bad_vibes:
                    keys_with_harps.append(kk)
                    orbits_with_harps.append(o)
                    
            else:
                print(kk+': HARP '+str(hh)+' NOT in orbit: ', o[0].strftime('%H-%M-%S'), o[1].strftime('%H-%M-%S'))
            
        #print(hh, keys.LON_FWT.values)

        if plot:
            fig, ax = plt.subplots(figsize=(12,6))      # define the size of the figure
            orangered = (1.0,0.27,0,1.0)                # create an orange-red color
            
            # define some style elements
            marker_style = dict(linestyle='', markersize=8, fillstyle='full',color=orangered,markeredgecolor=orangered)
            text_style = dict(fontsize=16, fontdict={'family': 'monospace'})
            ax.tick_params(labelsize=14)
            ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))

            #Correcting to get the radial (vs. line-of-sight) unsigned flux.
            the_brad = (keys.USFLUX)/np.cos(np.deg2rad(keys.LON_FWT))/1e22
            the_blos = (keys.USFLUX)/1e22
            #the_err = (keys.ERRVF)/np.cos(keys.LON_FWT)/1e22
            ax.plot(t_rec, the_brad,'o',**marker_style, label='B_rad')
            ax.plot(t_rec, the_blos, 'o', color='blue', label='B_los')

            #ax.errorbar(t_rec, the_brad, yerr=the_err, linestyle='', color=orangered)
            
            # ascribe the data to the axes
            #ax.plot(t_rec, (keys.USFLUX)/(1e22),'o',**marker_style)
            #ax.errorbar(t_rec, (keys.USFLUX)/(1e22), yerr=(keys.ERRVF)/(1e22), linestyle='', color=orangered)

            ax.legend()
            plt.savefig('./figures_etc/'+kk+'_HARP_'+str(hh)+'_unsigned_flux_rad.png')
            plt.close()
            

            fig, ax = plt.subplots(figsize=(12,6))      # define the size of the figure

            
            # define some style elements
            marker_style = dict(linestyle='', markersize=8, fillstyle='full',color='darkgreen', markeredgecolor='darkgreen')
            text_style = dict(fontsize=16, fontdict={'family': 'monospace'})
            #ax.tick_params(labelsize=14)
            #ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
            
            # ascribe the data to the axes
            ax.plot(t_rec, (keys.TOTUSJZ),'o',**marker_style)
            ax.scatter(t_rec, (keys.TOTUSJZ), linestyle='', color='darkgreen')

            plt.savefig('./figures_etc/'+kk+'_HARP_'+str(hh)+'_unsigned_helicity.png')
            plt.close()

In [ ]:
"""
This takes the keys_with_harps list and then, for each key, gets the result files. For each result file, the HARP parameters 
of interest (along with other DEM output parameters) for each result file are extracted. It then makes lists of said parameters and 
averages them per-key. 

It can be done for either flare or non-flare intervals, or all. It determines this by actually fetching the flarelist and checking the 
time for each interval if relevant. This makes it slow. 

Thus, later, plots of average parameters can be generated using the exported lists.

"""

importlib.reload(haa)
importlib.reload(ana)

re_query=True
#interval_type='non-flare'
#interval_type='all'
interval_type='flare'

with open('all_targets.pickle', 'rb') as f:
    all_targets = pickle.load(f)

c = haa.get_drms_client()
    
files_with_usflx=[] 
em10s_=[]
usflxs_=[]
key_avg_usflxs=[]
key_avg_em10, key_avg_em7, key_avg_em5 = [], [], []
key_avg_powerlaws = []
key_avg_peaks = []

for key in keys_with_harps:
    
    harps, orbits = haa.get_harp_and_times(key, all_targets)

    res_files, tworegion = ana.get_key_resultfiles(key, fromhome=True, 
                            keydict=all_targets,
                            withparams=True)
    
    if tworegion:
        for i in range(0, 2):
            hh = harps[i]
            rfs = res_files[i]
            print('HARP: ', hh)
            if re_query:
                files_with_usflx, usflxs, em10s, em7s, em5s, powerlaws, peaks = haa.save_usflux(rfs, hh, c, interp_seconds=20, 
                                                                          interval_type=interval_type, plot=True, label=key)
            else:
                files_with_usflx, usflxs, em10s, em7s, em5s, powerlaws, peaks = haa.get_usflux(rfs, interval_type=interval_type)
                
            if usflxs:
                files_with_usflx.extend(files_with_usflx)
                em10s_.extend(em10s)
                usflxs_.extend(usflxs)
                key_avg_em10.append(np.mean(em10s))
                key_avg_em7.append(np.mean(em7s))
                key_avg_em5.append(np.mean(em5s))
                key_avg_usflxs.append(np.nanmean(usflxs, axis=0))
                key_avg_powerlaws.append(np.nanmean(powerlaws, axis=0))
                key_avg_peaks.append(np.nanmean(peaks))
            
    else:
        hh = harps[0]
        print('HARP: ', hh)
        if re_query:
            files_with_usflx, usflxs, em10s, em7s, em5s, powerlaws, peaks = haa.save_usflux(res_files, hh, c, interp_seconds=20, 
                                                                      interval_type=interval_type, plot=True, label=key)
        else:
            files_with_usflx, usflxs, em10s, em7s, em5s, powerlaws, peaks = haa.get_usflux(res_files, interval_type=interval_type)
            
        if usflxs:
            files_with_usflx.extend(files_with_usflx)
            em10s_.extend(em10s)
            usflxs_.extend(usflxs)
            key_avg_em10.append(np.mean(em10s))
            key_avg_em7.append(np.mean(em7s))
            key_avg_em5.append(np.mean(em5s))
            key_avg_usflxs.append(np.nanmean(usflxs, axis=0))
            key_avg_powerlaws.append(np.nanmean(powerlaws, axis=0))
            key_avg_peaks.append(np.nanmean(peaks))
            

In [ ]:
# import visualize_dem_results as viz
# oops = '/Users/jmdunca2/do-dem/initial_dem_2sep15/region_0//02-22-30_02-31-10/02-22-30_02-31-10_5.6_7.2_02-sep-15_MC_DEM_result.pickle'

# viz.get_DEM_params(oops, save_params_file=True)

In [ ]:
#Plotting magnetic parameters vs. PEAKS

avg_usflxs_arr = np.array(key_avg_usflxs)
key_avg_peaks_arr = np.array(key_avg_peaks)

fig, axes = plt.subplots(5,1, figsize=(8,20)) 

xlabelz = ['(log) Total Unsigned Flux - LOS', '(log) Total Unsigned Flux - radial', 
           'Mean ALP', ' Mean Current Helicity', '(log) Total Unsigned Helicity']

for i in range(0,len(axes)):
    ax=axes[i]
    if i in [0,1,4]:

        ax.scatter(np.log10(avg_usflxs_arr[:,i]), key_avg_peaks_arr, marker='x', color='purple', label='DEM Peak')
        ax.set_ylabel('log(T)')
        #lower
        m, b = np.polyfit(np.log10(avg_usflxs_arr[:,i]), key_avg_peaks_arr, 1)
        fity = np.array(np.log10(avg_usflxs_arr[:,i]))*m + b
        ax.plot(np.log10(avg_usflxs_arr[:,i]), fity, linestyle='dotted', color='purple', label='linear fit, slope: '+str(round(m,3)))


        if i in [0,1]:
            ax.set_xlim([21,23])

    else:
        ax.scatter(avg_usflxs_arr[:,i], key_avg_peaks_arr, marker='x', color='purple', label='DEM Peak')
        ax.set_ylabel('log(T)')
        # m, b = np.polyfit(key_avg_em10, avg_usflxs_arr[:,i], 1)
        # fity = np.array(key_avg_em10)*m + b
        # ax.plot(key_avg_em10, fity, linestyle='dotted', color='black', label='linear fit, slope: '+str(round(m,3)))
        #ax.set_yscale('log')
        
    ax.set_xlabel(xlabelz[i])     
    ax.legend()

axes[0].set_title(interval_type)


#axes[0].set_ylim([1e20, 1e23])
#axes[0].set_xlim([1e17,0.5e23])

#plt.savefig('Peaks_vs_SHARP_params_'+interval_type+'rm_farlimbs.png')

In [ ]:
#Plotting magnetic parameters vs. powerlaws

avg_usflxs_arr = np.array(key_avg_usflxs)
key_avg_powerlaws_arr = np.array(key_avg_powerlaws)

fig, axes = plt.subplots(5,1, figsize=(8,20)) 

xlabelz = ['(log) Total Unsigned Flux - LOS', '(log) Total Unsigned Flux - radial', 
           'Mean ALP', ' Mean Current Helicity', '(log) Total Unsigned Helicity']

for i in range(0,len(axes)):
    ax=axes[i]
    if i in [0,1,4]:

        ax.scatter(np.log10(avg_usflxs_arr[:,i]), key_avg_powerlaws_arr[:,0], marker='x', color='violet', label='Low Powerlaw')
        ax.scatter(np.log10(avg_usflxs_arr[:,i]), -1*key_avg_powerlaws_arr[:,1], marker='x', color='indianred', label='High Powerlaw')
        ax.set_ylabel('Index')
        #lower
        m, b = np.polyfit(np.log10(avg_usflxs_arr[:,i]), key_avg_powerlaws_arr[:,0], 1)
        fity = np.array(np.log10(avg_usflxs_arr[:,i]))*m + b
        ax.plot(np.log10(avg_usflxs_arr[:,i]), fity, linestyle='dotted', color='violet', label='linear fit, slope: '+str(round(m,3)))

        #upper
        m, b = np.polyfit(np.log10(avg_usflxs_arr[:,i]), -1*key_avg_powerlaws_arr[:,1], 1)
        fity = np.array(np.log10(avg_usflxs_arr[:,i]))*m + b
        ax.plot(np.log10(avg_usflxs_arr[:,i]), fity, linestyle='dotted', color='indianred', label='linear fit, slope: '+str(round(m,3)))

        if i in [0,1]:
            ax.set_xlim([21,23])

    else:
        ax.scatter(avg_usflxs_arr[:,i], key_avg_powerlaws_arr[:,0], marker='x', color='violet', label='Low Powerlaw')
        ax.scatter(avg_usflxs_arr[:,i], -1*key_avg_powerlaws_arr[:,1], marker='x', color='indianred', label='High Powerlaw')
        ax.set_ylabel('Index')
        # m, b = np.polyfit(key_avg_em10, avg_usflxs_arr[:,i], 1)
        # fity = np.array(key_avg_em10)*m + b
        # ax.plot(key_avg_em10, fity, linestyle='dotted', color='black', label='linear fit, slope: '+str(round(m,3)))
        #ax.set_yscale('log')
        
    ax.set_xlabel(xlabelz[i])     
    ax.legend()

axes[0].set_title(interval_type)


#axes[0].set_ylim([1e20, 1e23])
#axes[0].set_xlim([1e17,0.5e23])

plt.savefig('Powerlaw_vs_SHARP_params_'+interval_type+'rm_farlimbs.png')

In [ ]:
avg_usflxs_arr = np.array(key_avg_usflxs)

fig, axes = plt.subplots(5,1, figsize=(8,20)) 

xlabelz = ['(log) Total Unsigned Flux - LOS', '(log) Total Unsigned Flux - radial', 
           'Mean ALP', ' Mean Current Helicity', '(log) Total Unsigned Helicity']

for i in range(0,len(axes)):
    ax=axes[i]
    if i in [0,1,4]:

        ax.scatter(np.log10(avg_usflxs_arr[:,i]), np.log10(key_avg_em10), marker='x', color='dodgerblue', label='Int. > 10 MK')
        ax.scatter(np.log10(avg_usflxs_arr[:,i]), np.log10(key_avg_em7), marker='x', color='darkgreen', label='Int. > 7 MK')
        ax.scatter(np.log10(avg_usflxs_arr[:,i]), np.log10(key_avg_em5), marker='x', color='palevioletred', label='Int. > 5 MK')
        ax.set_ylabel('(log) EM [cm^-5]')
        #10 MK
        m, b = np.polyfit(np.log10(avg_usflxs_arr[:,i]), np.log10(key_avg_em10), 1)
        fity = np.array(np.log10(avg_usflxs_arr[:,i]))*m + b
        ax.plot(np.log10(avg_usflxs_arr[:,i]), fity, linestyle='dotted', color='dodgerblue', label='linear fit, slope: '+str(round(m,3)))

        #7 MK
        m, b = np.polyfit(np.log10(avg_usflxs_arr[:,i]), np.log10(key_avg_em7), 1)
        fity = np.array(np.log10(avg_usflxs_arr[:,i]))*m + b
        ax.plot(np.log10(avg_usflxs_arr[:,i]), fity, linestyle='dotted', color='darkgreen', label='linear fit, slope: '+str(round(m,3)))

        #5 MK
        m, b = np.polyfit(np.log10(avg_usflxs_arr[:,i]), np.log10(key_avg_em5), 1)
        fity = np.array(np.log10(avg_usflxs_arr[:,i]))*m + b
        ax.plot(np.log10(avg_usflxs_arr[:,i]), fity, linestyle='dotted', color='palevioletred', label='linear fit, slope: '+str(round(m,3)))
        
        if i in [0,1]:
            ax.set_xlim([21,23])

    else:
        ax.scatter(avg_usflxs_arr[:,i], key_avg_em10, marker='x')
        ax.set_ylabel('EM [cm^-5]')
        # m, b = np.polyfit(key_avg_em10, avg_usflxs_arr[:,i], 1)
        # fity = np.array(key_avg_em10)*m + b
        # ax.plot(key_avg_em10, fity, linestyle='dotted', color='black', label='linear fit, slope: '+str(round(m,3)))
        ax.set_yscale('log')
        
    ax.set_xlabel(xlabelz[i])     
    ax.legend()

axes[0].set_title(interval_type)


#axes[0].set_ylim([1e20, 1e23])
#axes[0].set_xlim([1e17,0.5e23])

plt.savefig('EM_vs_SHARP_params_'+interval_type+'rm_farlimbs.png')

In [ ]:
np.log(key_avg_em10)

In [ ]:
usflxs_arr = np.array(usflxs_)

fig, axes = plt.subplots(4,1, figsize=(6,16)) 

xlabelz = ['Total Unsigned Flux', 'Mean ALP', 'Mean Current Helicity', 'Total Unsigned Helicity']

for i in range(0,len(axes)):
    ax=axes[i]
    ax.scatter(np.log10(em10s_), np.log10(usflxs_arr[:,i]), marker='x')
    #ax.set_yscale('log')
    #ax.set_xscale('log')
    ax.set_xlabel('Emission measure integrated over 10 MK [cm^-5]')
    ax.set_ylabel(xlabelz[i])   

    m, b = np.polyfit(np.log10(em10s_), np.log10(usflxs_arr[:,i]), 1)
    fity = np.array(np.log10(em10s_))*m + b
    ax.plot(np.log10(em10s_), fity, linestyle='dotted', color='black', label='linear fit, slope: '+str(round(m,3)))

axes[0].set_title('Non-flaring times')


#axes[0].set_ylim([1e20, 1e23])
#axes[0].set_xlim([1e17,0.5e23])

In [ ]:
usflxs_arr = np.array(usflxs_)

fig, axes = plt.subplots(4,1, figsize=(6,16)) 

xlabelz = ['Total Unsigned Flux', 'Mean ALP', 'Mean Current Helicity', 'Total Unsigned Helicity']

for i in range(0,len(axes)):
    ax=axes[i]
    ax.scatter(em10s_, usflxs_arr[:,i], marker='x')
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xlabel('Emission measure integrated over 10 MK [cm^-5]')
    ax.set_ylabel(xlabelz[i])           

In [ ]:
np.array(usflux)
t_rec
every_30s

In [ ]:
fig, ax = plt.subplots(figsize=(8,7))      # define the size of the figure
orangered = (1.0,0.27,0,1.0)                # create an orange-red color

# define some style elements
marker_style = dict(linestyle='', markersize=8, fillstyle='full',color=orangered,markeredgecolor=orangered)
text_style = dict(fontsize=16, fontdict={'family': 'monospace'})
ax.tick_params(labelsize=14)
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))

# ascribe the data to the axes
ax.plot(t_rec, (keys.USFLUX)/(1e22),'o',**marker_style)
ax.errorbar(t_rec, (keys.USFLUX)/(1e22), yerr=(keys.ERRVF)/(1e22), linestyle='',color=orangered)

# format the x-axis with universal time
locator = AutoDateLocator()
locator.intervald[HOURLY] = [12] # only show every 3 hours
formatter = DateFormatter('%H')
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

# set yrange 
#ax.set_ylim([2.4,2.8])

# label the axes and the plot
ax.set_xlabel('time in UT',**text_style)
ax.set_ylabel('maxwells x 1e22',**text_style)
ax.set_title('total unsigned flux starting at '+str(t_rec[0])+' UT',**text_style) # annotate the plot with a start time


In [ ]:
obsids=[]
for id in id_dirs:
    evt_data, hdr, obsid = nu.return_submap(datapath=id, fpm='A', return_evt_hdr=True, return_obsid=True)
    obsids.append(obsid)
    time0, time1 = [nuutil.convert_nustar_time(hdr['TSTART']), nuutil.convert_nustar_time(hdr['TSTOP'])]
    timerange = [time0, time1]
    print(timerange[0].strftime('%H-%M-%S'), timerange[1].strftime('%H-%M-%S'))
    